<h1>Spotted samples</h1>
<h3>Analyze peak intensity for spotted MSI data</h3>

In [ ]:
#load the code. Since it's specialized ipython notebook code, use '%run' rather than 'import'
%run Arrayed_Analysis_Tools.ipy

In [ ]:
#log into OpenMSI.nersc.gov
openMSIsession = login()


In [ ]:
#show a file selector
openMSIsession.fileSelector() #if this is not the last statement of the cell,
                              #you'll need to call IPython.display.display() on this

In [ ]:
#generates an ArrayedImage from the specificied ions.
myIons = [1241.3, 917.45, 1143] #nimzyme

img=openMSIsession.getArrayedImage(myIons,0.3,massRangeReductionStrategy=PeakArea())
# MassReductionStrategies available right now are PeakArea() PeakHeight() and AreaNearPeak()
# PeakArea() is the default


In [ ]:
#optional: check what the image looks likes now

plt.imshow(img.baseImage,cmap='jet_r',clim=(0.0,np.amax(img.baseImage)/2)) #get rid of the /2 to see a wider range, or divide
                                                                           #by a bigger number if you want a narrower range
plt.colorbar() 
plt.show()

In [ ]:
#define spot centers as a trapezoid.

Nrows = 16
Ncols = 24

img.roughPosition(Nrows,Ncols,colormap='jet_r')

In [ ]:
#automagically optimize the spot centers to correspond to the actual spots on the image
img.optimizeSpots(verbose=True)

#if you don't want spots to move off the grid unless they actually find a spot, set a minimumScore.
#(this is useful is there's missing spots from your grid)
#if you want to find out what a reasonable number for a minimumScore is, try running the optimization with verbose=True
#conversely, if you want to make /sure/ that there is a spot with a score of /at least/ minimumScore,
#you can choose to set raiseExceptions=True

#img.optimizeSpots(minimumScore=10000,halfboxsize=1) #this command works well for the 20120913_nimzyme.h5 image

In [ ]:
#check the positions of the spots and manually adjust them if need be
img.fineTunePosition(colormap='jet_r')

In [ ]:
#Optional: Save the ArrayedImage into a pickle file.
import pickle
pickle.dump(img, open("saved_image","wb"))

In [ ]:
#Optional: Load an ArrayedImage from a pickle file. This way you can work off-line
import pickle
img=pickle.load(open("saved_image","rb"))
print img

In [ ]:
#You'll need to call this function. It returns a list of spots (where each spot is a list of pixels),
#which is also stored inside the object.
spots=img.generateSpotList(integrationRadius=2)


In [ ]:
#Optional: If you want to see a visual respresenation of which pixels are taken into account when
#calculating the result table. If you don't pass it an explicit spotList it will use the stored spotList
maskedimg=img.generateMaskedImage()

plt.imshow(maskedimg)
plt.show()

In [ ]:
#Write results to a file
#if you don't pass it an explicit spotList it will use the spot set stored in the ArrayedImage
img.writeResultTable(open('export.tab','wb'))